<center><h1> Final Assignment <h1/><center/>
<hr>

**1. Description of the problem and a discussion of the background**

A chef from New York owns a trendy upscale restaurant in the city's Upper East Side. Next to the fantastic food and service, the owner attributes the restaurant's prime location as one of the prime drivers of success. 
Following the New York location, the chef is planning an expansion into other major US cities, in particular Chicago and San Francisco. Before the chef starts looking for specific locations in those cities, they want to know which neighborhoods in those particular cities are the most similar to New York's Upper East Side, both culturally and economically, in order to guarantee the highest chances of success for the expansion. 

The following model can be used by business owners in various sectors to find the right locations for new business expansions and openings, be it in the same city or in a new city. 




**2. A description of the data and how it will be used to solve the problem**

In order to find out which neighbordhoods in Chicago and San Francisco are most similar to New York's Upper East Side, we must perform a cluster analysis (K-Means Clustering). The problem we have falls within the branch of unsupervised machine learning, and we will be dealing with unlabeled data. 
The first data requirement is a list of all neighborhoods in the cities of Chicago and San Francisco, as well as our Manhattan Neighborhood, the Upper East Side. 

The Chicago data is available on the following link: <br>
CSV file: https://data.cityofchicago.org/api/views/y6yq-dbs2/rows.csv?accessType=DOWNLOAD <br>
All formats: https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Neighborhoods/bbvz-uum9

The San Francisco data is available on the following link: <br>
CSV file: https://data.sfgov.org/api/views/6ia5-2f8k/rows.csv?accessType=DOWNLOAD <br>
All formats: https://data.sfgov.org/Geographic-Locations-and-Boundaries/SF-Find-Neighborhoods/pty2-tcw4

The New York data is available on the following link: <br>
GeoJSON file: https://geo.nyu.edu/catalog/nyu-2451-34572 <br>

The Chicago and San Francisco data are csv files, which be easily read on a pandas dataframe. 

Let's import Pandads and take a look at them:

In [5]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

print('Libraries imported.')

Libraries imported.



The Chicago dataset contains the variables 'the_geom','PRI_NEIGH' which is our variable of interest, 'SEC_NEIGH', 'SHAPE_AREA', and 'SHAPE_LEN' and has a total of 98 rows, one for each neighborhood. 

In [3]:
Chicago_df=pd.read_csv('Chicago_Neighborhoods.csv')
Chicago_df.head()

,the_geom,PRI_NEIGH,SEC_NEIGH,SHAPE_AREA,SHAPE_LEN
0,MULTIPOLYGON (((-87.60670812560372 41.81681377...,Grand Boulevard,BRONZEVILLE,4.849250e+07,28196.837157
1,MULTIPOLYGON (((-87.62760697485348 41.87437097...,Printers Row,PRINTERS ROW,2.162138e+06,6864.247156
2,MULTIPOLYGON (((-87.66706868914602 41.88885187...,United Center,UNITED CENTER,3.252051e+07,23101.363745
3,MULTIPOLYGON (((-87.65833494805533 41.92166144...,Sheffield & DePaul,SHEFFIELD & DEPAUL,1.048259e+07,13227.049745
4,MULTIPOLYGON (((-87.74059567509266 41.88782316...,Humboldt Park,HUMBOLDT PARK,1.250104e+08,46126.751351


In [4]:
Chicago_df.shape

(98, 5)


The San Francisco dataset contains the variables 'LINK','the_geom', and 'name' which is our variable of interest, and has 117 rows, one for each neighborhood. 

In [10]:
SanFrancisco_df=pd.read_csv('SFFind_Neighborhoods.csv')
SanFrancisco_df.head()

,LINK,the_geom,name
0,"http://en.wikipedia.org/wiki/Sea_Cliff,_San_Fr...",MULTIPOLYGON (((-122.49345526799993 37.7835181...,Seacliff
1,NaN,MULTIPOLYGON (((-122.48715071499993 37.7837854...,Lake Street
2,http://www.nps.gov/prsf/index.htm,MULTIPOLYGON (((-122.47758017099994 37.8109931...,Presidio National Park
3,NaN,MULTIPOLYGON (((-122.47241052999993 37.7873465...,Presidio Terrace
4,http://www.sfgate.com/neighborhoods/sf/innerri...,MULTIPOLYGON (((-122.47262578999994 37.7863148...,Inner Richmond


In [11]:
SanFrancisco_df.shape

(117, 3)


The New York data is a GeoJSON file, from which we can find the neighborhood name under 'features.properties.name' and our geospatial coordinates under 'features.geometry.coordinates'. 


In [12]:
with open('nyu-2451-34572-geojson.json') as json_data:
    newyork_data=json.load(json_data)
    
neighborhood_data=newyork_data['features']
neighborhood_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Having these files, we must then filter out the neighborhoods, and assign them with a prefix to label them with their respective city.

 
To explore cultural similarity, we will be using the FourSquare API to find out what types of venues are popular in each of the neighborhoods and then compare them with each other. Therefore, we will need the Latitude and Longitude values of the neighborhoods. For Chicago and San Francisco we can get those through the Google Maps API and add them to our Dataframe. 

Now we are ready to call the FourSqaure API and connect to the search endpoint in the venues group. This will return a JSON file, from which we extract the info on the venue category and place it in a pandas dataframe, along with the neighborhood name. For the KMeans clustering we first calculate the mean frequencies of each venue category in a particular neighborhood, compute a measure of similarity (dissimilarity) of the neighborhood to each of the K cluster centers (centroids), and then assign it to the one closest to it. Recalculating the K centroids based on the mean of data points within that cluster and reallocating data points based on distance will lead to the algorithm giving us a total of K different clusters that have maximum similarity within themselves, but maximum dissimilarity among each other. 

Once we have our clusters, we can focus on the one containing New York's Upper East side and the culturally similar Chicago and San Francisco neighborhoods. To find neighborhoods which have a high proportion of high income households, similar to the Upper East Side, we need to further divide this cluster based on Median Household Income. This is the appropriate measure, given that the median household income is is generally less affected by outliers (i.e. very high income individuals) than the mean income and can give a better indication of the average levels of income for the residents in the neighborhood. I was unable to find a dataset that cointains each neighborhood and their median household income online for free, therefore, we will be using a dataset that lists the median household income by postal code. Please note that the data is from 2006-2010, and processes like gentrification may have significantly affected this data (especially in low-income areas around the city center).

The dataset can be found on the following link: https://www.psc.isr.umich.edu/dis/census/Features/tract2zip/

We can see that the file has four columns: 'Zip' for ZIP code, 'Median' for median income, 'Mean' for mean income, and 'Pop' for population. In total. the file has information on 32,634 postal codes in the United States. 

In [13]:
zip_income_df=pd.read_csv('MedianZIP.csv')
zip_income_df.head()

,Zip,Median,Mean,Pop
0,1001,"56,663","66,688","16,445"
1,1002,"49,853","75,063","28,069"
2,1003,"28,462","35,121","8,491"
3,1005,"75,423","82,442","4,798"
4,1007,"79,076","85,802","12,962"


In [14]:
zip_income_df.shape

(32634, 4)

In order to map a median household income to each neighborhood, we must first find their postal code. It is to mention though that there might be some overlap between neighborhoods, or that some neighborhoods contain several postal codes and we will only be using the one our Latitude and Longitude data falls on. The postal code, again, can be found through the Google Maps API. 

Having the postal code allocated to each of the neighborhoods in our relevant cluster, we can get the information on the median household income. Now we can conduct another KMeans cluster analysis within the relvant cluster, based on income.  

The appropriate neighborhoods to open the restaurants in Chicago and San Francisco will be the ones that, again, share a cluster with the Upper East Side. To visualize our results, it is best to use a Choropleth map, coloring the areas based on suitability, from unsuitable, to culturally similar, to culturally and economically similar. Culturally and economically similar locations are optimal and will be colored in red. A choropleth map requires a GeoJSON file. 

The GeoJSON file of Chicago neighborhoods can be downloaded here:  <br>
https://data.cityofchicago.org/api/geospatial/bbvz-uum9?method=export&format=GeoJSON <br>
The GeoJSON file of San Francisco neighborhoods can be downloaded here: <br>
https://data.sfgov.org/api/geospatial/pty2-tcw4?method=export&format=GeoJSON <br>

For the Chicago neighborhoods, the key (neighborhood name) can be found under 'features.properties.pri_neigh' and for the San Francisco neighborhoods, the key can be found under 'features.properties.name'.



In [18]:
import json
with open('cbn.geojson') as f:
    chicago_boundaries=json.load(f)

for feature in chicago_boundaries['features']:
   print(feature['properties'])

{'pri_neigh': 'Grand Boulevard', 'sec_neigh': 'BRONZEVILLE', 'shape_area': '48492503.1554', 'shape_len': '28196.837157'}
{'pri_neigh': 'Printers Row', 'sec_neigh': 'PRINTERS ROW', 'shape_area': '2162137.97139', 'shape_len': '6864.247156'}
{'pri_neigh': 'United Center', 'sec_neigh': 'UNITED CENTER', 'shape_area': '32520512.7053', 'shape_len': '23101.363745'}
{'pri_neigh': 'Sheffield & DePaul', 'sec_neigh': 'SHEFFIELD & DEPAUL', 'shape_area': '10482592.2987', 'shape_len': '13227.049745'}
{'pri_neigh': 'Humboldt Park', 'sec_neigh': 'HUMBOLDT PARK', 'shape_area': '125010425.593', 'shape_len': '46126.751351'}
{'pri_neigh': 'Garfield Park', 'sec_neigh': 'GARFIELD PARK', 'shape_area': '89976069.5947', 'shape_len': '44460.91922'}
{'pri_neigh': 'North Lawndale', 'sec_neigh': 'NORTH LAWNDALE', 'shape_area': '89487422.0244', 'shape_len': '44959.459663'}
{'pri_neigh': 'Little Village', 'sec_neigh': 'LITTLE VILLAGE', 'shape_area': '127998297.819', 'shape_len': '49904.04003'}
{'pri_neigh': 'Armour S

In [16]:
with open('SF.geojson') as f2:
    sf_boundaries=json.load(f2)

sf_boundaries['features'][0]
for feature in sf_boundaries['features']:
   print(feature['properties'])

{'link': 'http://en.wikipedia.org/wiki/Sea_Cliff,_San_Francisco,_California', 'name': 'Seacliff'}
{'link': None, 'name': 'Lake Street'}
{'link': 'http://www.nps.gov/prsf/index.htm', 'name': 'Presidio National Park'}
{'link': None, 'name': 'Presidio Terrace'}
{'link': 'http://www.sfgate.com/neighborhoods/sf/innerrichmond/', 'name': 'Inner Richmond'}
{'link': None, 'name': 'Sutro Heights'}
{'link': None, 'name': 'Lincoln Park / Ft. Miley'}
{'link': 'http://www.sfgate.com/neighborhoods/sf/outerrichmond/', 'name': 'Outer Richmond'}
{'link': 'http://www.sfgate.com/neighborhoods/sf/goldengatepark/', 'name': 'Golden Gate Park'}
{'link': 'http://en.wikipedia.org/wiki/Neighborhoods_in_San_Francisco#Presidio_Heights', 'name': 'Presidio Heights'}
{'link': None, 'name': 'Laurel Heights / Jordan Park'}
{'link': 'http://en.wikipedia.org/wiki/Lone_Mountain,_San_Francisco,_California', 'name': 'Lone Mountain'}
{'link': 'http://en.wikipedia.org/wiki/Anza_Vista,_San_Francisco', 'name': 'Anza Vista'}
{'l